In [4]:
import os.path
import random
import numpy as np
from PIL import Image

import torch
import torchvision.transforms as transforms

from tqdm import tqdm

In [23]:
transform_list = []
resize = True
method = Image.NEAREST
normalize = False

osize = [128, 128]
if resize:
    transform_list.append(transforms.Resize(osize, method))


transform_list.append(transforms.RandomHorizontalFlip(p=0.5))

transform_list += [transforms.ToTensor()]

if normalize:
    transform_list += [transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5)
                                            )]

transform_label = transforms.Compose(transform_list)

/home/as3ek/.conda/envs/hist/lib/python3.8/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [37]:
label_path = "/scratch/as3ek/github/HistoMask/data/segvae/humanparsing/seg_deploy/2500_1.png"

In [40]:
label = Image.open(label_path)

In [41]:
np.unique(label)

array([ 0,  2,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16], dtype=uint8)

In [42]:
label_tensor = transform_label(label) * 255.0

In [43]:
torch.unique(label_tensor)

tensor([ 0.,  2.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16.])

In [62]:
label_path_lizard = "/scratch/as3ek/github/HistoMask/data/segvae/lizard/classes/consep_1__0_372.png"

In [63]:
label_lizard = Image.fromarray(np.array(Image.open(label_path_lizard)).astype(np.uint8))

In [64]:
np.unique(label_lizard)

array([0, 2, 3, 6], dtype=uint8)

In [65]:
label_lizard_tensor = transform_label(label_lizard) * 255.0

In [66]:
torch.unique(label_lizard_tensor)

tensor([0., 2., 3., 6.])